In [ ]:
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
    
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

import skrf as rf
skrf_color_names = ['blue','red','green','orange','purple','pink','grey']
skrf_colors = dict(zip(skrf_color_names, rf.plotting.colors()))

In [ ]:
file = open('i2c-eeprom.scopesession', 'r')
scopesession = load(file, Loader=Loader)

In [ ]:
waveform_1 = np.fromfile('.\i2c-eeprom_data\scope_1_waveforms\waveform_2\channel_1.bin', dtype='<f')#, count=100000)
waveform_2 = np.fromfile('.\i2c-eeprom_data\scope_1_waveforms\waveform_2\channel_2.bin', dtype='<f')#, count=100000)

In [ ]:
fig, axes = plt.subplots(figsize=(10,4), dpi=110, nrows=2, sharex=True, sharey=True, constrained_layout=True)

# Plot Config
samples_per_symbol = 250
#num_symbols = int(0.5* len(waveform_1)/ samples_per_symbol)
trigger = True

y_trigger_padding = int(samples_per_symbol*5)
y_trigger_start = np.where(waveform_1 < (np.max(waveform_1)*0.5))[0][0]
y_trigger_end   = np.where(waveform_1 < (np.max(waveform_1)*0.5))[0][-1]


y_start =  y_trigger_start - min(len(waveform_1[0:y_trigger_start]), y_trigger_padding) if trigger else 0
y_end   =  y_trigger_end   + min(len(waveform_1[y_trigger_end:]),    y_trigger_padding) if trigger else len(waveform_1)
y_range =  y_end-y_start

# Plot channels
x  = range(y_start, y_end)
y1 = waveform_1[y_start:y_end]
y2 = waveform_2[y_start:y_end]

axes[0].plot(x, y1, skrf_colors['blue'])
axes[1].plot(x, y2, skrf_colors['red'])

axes[0].vlines(min(len(waveform_1[0:y_trigger_start]), y_trigger_padding), -1,3)
axes[0].vlines(y_trigger_end - y_start, -1,3)

# Configure Plot
fig.suptitle("Oscilloscope Channels")
fig.supylabel("Volts")
fig.supxlabel("Time (Samples)")

xloc = mpl.ticker.MultipleLocator(base=5000)
yloc = mpl.ticker.MultipleLocator(base=1)

ymin, ymax = axes[0].get_ylim()
for ax in axes:
    ax.xaxis.set_major_locator(xloc)
    ax.yaxis.set_major_locator(yloc)
#    ax.xaxis.grid()
    ax.set_ylim(ymin,ymax)

plt.show()

In [ ]:
# resolution number of pixels divided by number of samples
samples_per_bin = (10*110) /(num_symbols*samples_per_symbol)
interp = 200

from scipy import interpolate

x_interp = np.arange(np.min(x), np.max(x), 1/interp)
f1 = interpolate.interp1d(x, y1)
y1_interp = f1(x_interp)

f2 = interpolate.interp1d(x, y2)
y2_interp = f2(x_interp)

In [ ]:
fig, axes = plt.subplots(figsize=(10,4), dpi=157*2, nrows=2, sharex=True, sharey=True, constrained_layout=True)

# Bins is just the figsize * dpi
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm
norm = PowerNorm(0.4, vmin=0,vmax=150)
norm = LogNorm(vmin=0.01,vmax=5000)
m_cmap = mpl.cm.get_cmap("viridis").copy()
m_cmap.set_bad(m_cmap.colors[0])
bins = (10*157, 4*157)

hist1, xedges1, yedges1, im1 = axes[0].hist2d(x_interp, y1_interp, bins=bins,cmap=m_cmap, norm=norm)
hist2, xedges2, yedges2, im2 = axes[1].hist2d(x_interp, y2_interp, bins=bins,cmap=m_cmap, norm=norm)

fig.suptitle("Oscilloscope Channels")
fig.supylabel("Volts")
fig.supxlabel("Time (Samples)")
plt.show()

In [ ]:
#rf.stylely()

# Plot histograms of voltage for each channel
hx1 = hist1.sum(axis=0)
hx2 = hist2.sum(axis=0)

fig, axes = plt.subplots(figsize=(8,4), dpi=157, ncols=2, constrained_layout=True)

# Normalize histograms to percentages
v1_norm = hx1#/np.sum(hx1)
v2_norm = hx2#/np.sum(hx2)

ch1_vrange = np.linspace(np.min(waveform_1), np.max(waveform_1), len(hx1))
ch2_vrange = np.linspace(np.min(waveform_2), np.max(waveform_2), len(hx2))

print(f'{np.max(v1_norm):.2f}')

axes[0].semilogy(ch1_vrange, v1_norm, skrf_colors['blue'], label="Ch 1")
axes[0].semilogy(ch2_vrange, v2_norm, skrf_colors['red'], label="Ch 2")

axes[1].hist(y1_interp, 100, density=True, histtype='step', cumulative=True, color=skrf_colors['blue'], label="Ch 1")
axes[1].hist(y2_interp, 100, density=True, histtype='step', cumulative=True, color=skrf_colors['red'], label="Ch 2")

axes[0].legend(loc="lower right")
axes[0].set_xlabel("Voltage")
axes[0].set_ylabel("Num Bins")
axes[0].set_title("Bin PDF")


axes[1].legend(loc="lower right")
axes[1].set_xlabel("Voltage")
axes[1].set_ylabel("Cumulative Likelihood")
axes[1].set_title("Voltage CDF")
axes[1].set_ylim((0,1.05))

plt.show()

In [ ]:
# Final Comparison Plot
fig, axes = plt.subplots(figsize=(10,10), dpi=157*2, nrows=4, sharex=True, sharey=True, constrained_layout=True)
axes[0].plot(x, y1, skrf_colors['blue'])
axes[1].hist2d(x_interp, y1_interp, bins=bins,cmap=m_cmap, norm=norm)
axes[1].imshow(grid, interpolation=interp_method)
axes[2].plot(x, y2, skrf_colors['red'])
axes[3].hist2d(x_interp, y2_interp, bins=bins,cmap=m_cmap, norm=norm)
axes[3].imshow(grid, interpolation=interp_method)

axes[0].set_title("Channel 1")
axes[2].set_title("Channel 2")

xloc = mpl.ticker.MultipleLocator(base=10000)
yloc = mpl.ticker.MultipleLocator(base=1)

for ax in axes:
    ax.xaxis.set_major_locator(xloc)
    ax.yaxis.set_major_locator(yloc)
    ax.set_ylim(ymin,ymax)

fig.suptitle("Oscilloscope Channels")
fig.supylabel("Volts")
fig.supxlabel("Time (Samples)")
plt.show()

In [ ]:
from scipy.stats import kde

fig, axes = plt.subplots(figsize=(10,10), dpi=157*2, nrows=4, sharex=True, sharey=True, constrained_layout=True)

k = kde.gaussian_kde([x,y1])
xi, yi = np.mgrid[np.min(x):np.max(x):bins[0]*1j, np.min(y1):np.max(y1):bins[1]*1j]
zi = k(np.vstack([xi.flatten(), yi.flatten()]))
 
# Make the plot
axes[0].pcolormesh(xi, yi, zi.reshape(xi.shape), shading='auto')
plt.show()